In [2]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_union
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pickle

In [3]:
data = pd.read_csv('WA_Fn-UseC_-Sales-Win-Loss.csv')

In [70]:
data.head()

,Opportunity Number,Supplies Subgroup,Supplies Group,Region,Route To Market,Elapsed Days In Sales Stage,Opportunity Result,Sales Stage Change Count,Total Days Identified Through Closing,Total Days Identified Through Qualified,Opportunity Amount USD,Client Size By Revenue,Client Size By Employee Count,Revenue From Client Past Two Years,Competitor Type,Ratio Days Identified To Total Days,Ratio Days Validated To Total Days,Ratio Days Qualified To Total Days,Deal Size Category
0,1641984,Exterior Accessories,Car Accessories,Northwest,Fields Sales,76,Won,13,104,101,0,5,5,0,Unknown,0.69636,0.113985,0.154215,1
1,1658010,Exterior Accessories,Car Accessories,Pacific,Reseller,63,Loss,2,163,163,0,3,5,0,Unknown,0.00000,1.000000,0.000000,1
2,1674737,Motorcycle Parts,Performance & Non-auto,Pacific,Reseller,24,Won,7,82,82,7750,1,1,0,Unknown,1.00000,0.000000,0.000000,1
3,1675224,Shelters & RV,Performance & Non-auto,Midwest,Reseller,16,Loss,5,124,124,0,1,1,0,Known,1.00000,0.000000,0.000000,1
4,1689785,Exterior Accessories,Car Accessories,Pacific,Reseller,69,Loss,11,91,13,69756,1,1,0,Unknown,0.00000,0.141125,0.000000,4


In [75]:
data.describe()

,Opportunity Number,Elapsed Days In Sales Stage,Sales Stage Change Count,Total Days Identified Through Closing,Total Days Identified Through Qualified,Opportunity Amount USD,Client Size By Revenue,Client Size By Employee Count,Revenue From Client Past Two Years,Ratio Days Identified To Total Days,Ratio Days Validated To Total Days,Ratio Days Qualified To Total Days,Deal Size Category
count,7.802500e+04,78025.000000,78025.000000,78025.000000,78025.000000,78025.000000,78025.000000,78025.000000,78025.000000,78025.000000,78025.000000,78025.000000,78025.000000
mean,7.653429e+06,43.595348,2.955732,16.728356,16.314348,91637.260750,1.620481,1.604050,0.303287,0.203063,0.488314,0.185048,3.436745
std,1.054848e+06,26.585600,1.497242,16.729593,16.562597,133161.029157,1.235796,1.210744,0.930290,0.364985,0.448077,0.340283,1.656890
min,1.641984e+06,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,6.900423e+06,19.000000,2.000000,4.000000,4.000000,15000.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,7.545569e+06,43.000000,3.000000,12.000000,12.000000,49000.000000,1.000000,1.000000,0.000000,0.000000,0.448000,0.000000,3.000000
75%,8.228329e+06,65.000000,3.000000,24.000000,24.000000,105099.000000,1.000000,1.000000,0.000000,0.197225,1.000000,0.188552,5.000000
max,1.009427e+07,210.000000,23.000000,208.000000,208.000000,1000000.000000,5.000000,5.000000,4.000000,1.000000,1.000000,1.000000,7.000000


In [33]:
data = data.fillna(0)

In [36]:
data.dtypes

Opportunity Number                           int64
Supplies Subgroup                           object
Supplies Group                              object
Region                                      object
Route To Market                             object
Elapsed Days In Sales Stage                  int64
Opportunity Result                          object
Sales Stage Change Count                     int64
Total Days Identified Through Closing        int64
Total Days Identified Through Qualified      int64
Opportunity Amount USD                       int64
Client Size By Revenue                       int64
Client Size By Employee Count                int64
Revenue From Client Past Two Years           int64
Competitor Type                             object
Ratio Days Identified To Total Days        float64
Ratio Days Validated To Total Days         float64
Ratio Days Qualified To Total Days         float64
Deal Size Category                           int64
dtype: object


## remove outliers using the inter quartile range

The interquartile range is a measure of where the “middle fifty” is in a data set. Where a range is a measure of where the beginning and end are in a set, an interquartile range is a measure of where the bulk of the values lie. That’s why it’s preferred over many other measures of spread (i.e. the average or median) when reporting things like school performance or SAT scores.

The interquartile range formula is the first quartile subtracted from the third quartile.

source: https://www.statisticshowto.datasciencecentral.com/probability-and-statistics/interquartile-range/

In [4]:
cont = ['Elapsed Days In Sales Stage', 'Sales Stage Change Count', 'Total Days Identified Through Closing',
       'Total Days Identified Through Qualified', 'Opportunity Amount USD', 'Ratio Days Identified To Total Days',
       'Ratio Days Validated To Total Days', 'Ratio Days Qualified To Total Days']

In [5]:
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

Opportunity Number                         1.327906e+06
Elapsed Days In Sales Stage                4.600000e+01
Sales Stage Change Count                   1.000000e+00
Total Days Identified Through Closing      2.000000e+01
Total Days Identified Through Qualified    2.000000e+01
Opportunity Amount USD                     9.009900e+04
Client Size By Revenue                     0.000000e+00
Client Size By Employee Count              0.000000e+00
Revenue From Client Past Two Years         0.000000e+00
Ratio Days Identified To Total Days        1.972250e-01
Ratio Days Validated To Total Days         1.000000e+00
Ratio Days Qualified To Total Days         1.885520e-01
Deal Size Category                         3.000000e+00
dtype: float64


In [6]:
no_outliers = data[~((data[cont] < (Q1 - 1.5 * IQR)) |(data[cont] > (Q3 + 1.5 * IQR))).any(axis=1)]
no_outliers.shape

(38827, 19)

## dropped lots of data - not recommended in a real project!

In [78]:
data.shape

(78025, 19)

In [79]:
no_outliers.head()

,Opportunity Number,Supplies Subgroup,Supplies Group,Region,Route To Market,Elapsed Days In Sales Stage,Opportunity Result,Sales Stage Change Count,Total Days Identified Through Closing,Total Days Identified Through Qualified,Opportunity Amount USD,Client Size By Revenue,Client Size By Employee Count,Revenue From Client Past Two Years,Competitor Type,Ratio Days Identified To Total Days,Ratio Days Validated To Total Days,Ratio Days Qualified To Total Days,Deal Size Category
43,4562313,Shelters & RV,Performance & Non-auto,Midwest,Fields Sales,78,Loss,4,35,35,70000,3,3,0,Unknown,0.259259,0.740741,0.000000,4
90,4931756,Exterior Accessories,Car Accessories,Midwest,Reseller,24,Won,4,8,8,10000,1,5,0,Unknown,0.000000,0.916667,0.083333,2
110,5036297,Garage & Car Care,Car Accessories,Pacific,Reseller,91,Loss,3,27,27,46504,3,5,0,Unknown,0.000000,1.000000,0.000000,3
111,5036371,Shelters & RV,Performance & Non-auto,Midwest,Reseller,24,Won,3,12,12,100000,4,1,0,Unknown,0.000000,1.000000,0.000000,5
126,5098144,Exterior Accessories,Car Accessories,Midwest,Reseller,24,Won,4,7,7,6406,1,1,0,Unknown,0.000000,0.614286,0.385714,1


## logistic regression model on data without outliers

In [7]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [8]:
final_data = MultiColumnLabelEncoder(columns = ['Supplies Subgroup', 'Supplies Group', 'Region', 'Route To Market', 
                                  'Opportunity Result', 'Competitor Type']).fit_transform(no_outliers)

In [96]:
final_data.head()

,Opportunity Number,Supplies Subgroup,Supplies Group,Region,Route To Market,Elapsed Days In Sales Stage,Opportunity Result,Sales Stage Change Count,Total Days Identified Through Closing,Total Days Identified Through Qualified,Opportunity Amount USD,Client Size By Revenue,Client Size By Employee Count,Revenue From Client Past Two Years,Competitor Type,Ratio Days Identified To Total Days,Ratio Days Validated To Total Days,Ratio Days Qualified To Total Days,Deal Size Category
43,4562313,8,2,1,0,78,0,4,35,35,70000,3,3,0,2,0.259259,0.740741,0.000000,4
90,4931756,2,0,1,2,24,1,4,8,8,10000,1,5,0,2,0.000000,0.916667,0.083333,2
110,5036297,3,0,4,2,91,0,3,27,27,46504,3,5,0,2,0.000000,1.000000,0.000000,3
111,5036371,8,2,1,2,24,1,3,12,12,100000,4,1,0,2,0.000000,1.000000,0.000000,5
126,5098144,2,0,1,2,24,1,4,7,7,6406,1,1,0,2,0.000000,0.614286,0.385714,1


In [81]:
final_data.to_csv('final_data.csv', index= False)

In [9]:
yVar = final_data['Opportunity Result']
xVar = final_data.loc[:, final_data.columns != 'Opportunity Result']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(xVar, yVar, test_size=0.2)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(31061, 18) (31061,)
(7766, 18) (7766,)


In [98]:
y_test.head()

14941    0
36866    1
10930    0
70434    0
30301    0
Name: Opportunity Result, dtype: int64

In [11]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        assert isinstance(X, pd.DataFrame)

        try:
            return X[self.columns]
        except KeyError:
            cols_error = list(set(self.columns) - set(X.columns))
            raise KeyError("The DataFrame does not include the columns: %s" % cols_error)

In [34]:
pipeline = Pipeline(steps = [
        ("features", make_union(
                ColumnSelector(list(xVar)),
                )),
                ("model",LogisticRegression(random_state=42))
])

pipeline.fit(X_train, y_train)

pipeline.score(X_test, y_test)

C:\Users\skype\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.78032449137265

In [24]:
y_pred = pipeline.predict(X_test)
pd.crosstab(y_test, y_pred, rownames=['Actual Result'], colnames=['Predicted Result'])

Predicted Result,0
Actual Result,
0,5970
1,1796


In [90]:
y_test.head()

14941    0
36866    1
10930    0
70434    0
30301    0
Name: Opportunity Result, dtype: int64

In [95]:
y_pred[1]

0

## logistic regression with outliers score was 0.77 so this is an improvement

## save the model to load in the next video

In [60]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(pipeline, open(filename, 'wb'))
X_test.to_csv('X_test.csv', index = False)
y_test.columns = ['Opportunity Result']
y_test.to_csv('y_test.csv', index= False)


In [61]:
y_test.head()

69221    0
69884    0
48931    0
16362    0
46223    1
Name: Opportunity Result, dtype: int64